In [1]:
pip install selenium

  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB 660.6 kB/s eta 0:00:15
   ---------------------------------------- 0.0/9.7 MB 487.6 kB/s eta 0:00:20
   ---------------------------------------- 0.1/9.7 MB 939.4 kB/s eta 0:00:11
   - -------------------------------------- 0.3/9.7 MB 1.6 MB/s eta 0:00:06
   - -------------------------------------- 0.4/9.7 MB 2.0 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/9.7 MB 2.6 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.7 MB 3.3 MB/s eta 0:00:03
   ----- ---------------------------------- 1.4/9.7 MB 3.8 MB/s eta 0:00:03
   ------- -------------------------------- 1.9/9.7 MB 4.6 MB/s eta 0:00:02
   ------------ --------------------------- 3.0/9.7 MB 6.5 MB/s eta 0:00:02
   -------------------- ------------------

In [70]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import re  # Import regular expressions

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the provided URL
driver.get("https://www.goldenscent.com/c/new.html?action=cat&id=243")

# Wait for the page to load
time.sleep(5)  # Adjust the wait time if needed

# Initialize an empty list to store unique hrefs
unique_hrefs = []

# Loop to collect unique hrefs for up to 200 perfumes
while len(unique_hrefs) < 200:
    # Locate the perfume links and extract their href attributes
    golden_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
    golden_hrefs = [link.get_attribute('href') for link in golden_links]

    # Collect unique hrefs
    for href in golden_hrefs:
        if href not in unique_hrefs:
            unique_hrefs.append(href)
        if len(unique_hrefs) >= 200:  # Stop once we have 200 unique hrefs
            break

    # Scroll down to load more products if needed
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Adjust the wait time if needed

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_hrefs):
    try:
        driver.get(href)  # Navigate to the perfume detail page using the href
        time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            # Check if there is a discount (look for 'savings' class)
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                # Extract the discounted price from the second span within 'savings'
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                # If there's no discount, use the regular price in 'special'
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            # Store the final price if found
            if final_price:
                product_info["السعر النهائي"] = int(final_price)  # Convert to integer for consistency
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)  
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)  
            product_info[label] = value  

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

# Save the DataFrame to a CSV file
golden_df.to_csv("golden_scent_perfumes.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()


Error on perfume 53: HTTPConnectionPool(host='localhost', port=61205): Read timed out. (read timeout=120)
Error on perfume 54: HTTPConnectionPool(host='localhost', port=61205): Read timed out. (read timeout=120)
Error on perfume 97: HTTPConnectionPool(host='localhost', port=61205): Read timed out. (read timeout=120)
Error on perfume 98: HTTPConnectionPool(host='localhost', port=61205): Read timed out. (read timeout=120)
                                                 الاسم  السعر النهائي  \
0                          فرزاتشي كريستال نوار برفيوم            662   
1             فريدريك مالي أو دو ماجنوليا أو دو تواليت           1570   
2        مونت بلانك باتشولي إنك دارك وودز أو دو برفيوم            760   
3             جفنشي لانترديت أبسولو أو دو برفيوم إنتنس            768   
4    دكتور فرانجيس فيرنزيه روسو نوبل مجموعة هدايا -...            430   
..                                                 ...            ...   
191  كار أروما مجموعة موزع الروائح وزيوت عطرية - أب...           

In [71]:
golden_df

,الاسم,السعر النهائي,الماركة,الجنس,نوع المنتج,شخصية عطرك,العائلة العطرية,الحجم,سنة الإصدار,نسبة التركيز,مقدمة العطر,قلب العطر,قاعدة العطر,المكونات,منطقة الاستخدام,ملمس,نوع البشرة,المكونات الرئيسية
0,فرزاتشي كريستال نوار برفيوم,662,فرزاتشي,نسائي,عطور,جذابة,زهرية شرقية,90مل,2004,برفيوم مركز,الفلفل، الزنجبيل، الهيل,جوز الهند، الجاردينيا، زهور البرتقال، الفاونيا,أخشاب الصندل، المسك، العنبر,NaN,NaN,NaN,NaN,NaN
1,فريدريك مالي أو دو ماجنوليا أو دو تواليت,1570,فريدريك مال,رجالي/نسائي,عطور,ديناميكية,حمضية,100 مل,2014,أو دو تواليت,الليمون، الأوراق الخضراء، برجموت كالابريا,كالوني، الياسمين، الباتشولي، نجيل الهند، الماج...,العنبر، أخشاب الأرز، الطحالب,NaN,NaN,NaN,NaN,NaN
2,مونت بلانك باتشولي إنك دارك وودز أو دو برفيوم,760,مونت بلانك,رجالي,عطور,محبة للطبيعة,خشبية,125 مل,2024,أو دو برفيوم,البرجموت,الحبر,الفانيليا، الباتشولي,NaN,NaN,NaN,NaN,NaN
3,جفنشي لانترديت أبسولو أو دو برفيوم إنتنس,768,جفنشي,نسائي,عطور,جذابة,زهرية شرقية,80 مل,2020,أو دو برفيوم انتنس,اللافندر، النيرولي، الهيل الجواتيمالي,زهور البرتقال، ياسمين سمباك، مسك الروم الهندي,الباتشولي، نجيل الهند، الرم، التبغ,NaN,NaN,NaN,NaN,NaN
4,دكتور فرانجيس فيرنزيه روسو نوبل مجموعة هدايا -...,430,دكتور فرانجيس فيرنزيه,NaN,مجموعة عطور منزلية,NaN,زهرية شرقية,NaN,NaN,NaN,NaN,NaN,NaN,البرتقال، زهور البرتقال، البنفسج، الفراولة، ال...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,كار أروما مجموعة موزع الروائح وزيوت عطرية - أب...,129,مجموعات حصرية,NaN,موزع للرائحة إلكتروني,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,بي دي كيه 312 سانت أونر أو دو برفيوم,893,بي دي كيه,رجالي/نسائي,عطور,حسية,زهرية شرقية,100 مل,2024,أو دو برفيوم,جذور حشيشة الملاك، الفلفل الوردي,حبوب التونكا، زهور البرتقال، الفلفل الوردي,العود التايلاندي، أمبروستار، أمبروكسان، المسك ...,NaN,NaN,NaN,NaN,NaN
193,لا بونير فرصة برفيوم,108,لا بونير,رجالي/نسائي,عطور,حسية,زهرية,100 مل,NaN,برفيوم مركز,"Fruits, Peaches","Jasmine, Lily Of The Valley, Patchouli","Caramel, Leather, Musk, Raspberry, Vanilla",NaN,NaN,NaN,NaN,NaN
194,مينيز ميموريز أوف وينتر لندن أو دو برفيوم - 30 مل,59,مينيز,نسائي,عطور,حسية,زهرية,30 مل,2024,أو دو برفيوم,الزبيب الأسود,الورد، الفريزيا,الباتشولي، الفانيليا، الأخشاب,الزبيب الأسود، الورد، الفريزيا، الباتشولي، الف...,NaN,NaN,NaN,NaN
